
<a id='basics'></a>
{{ header }}

# Essential basic functionality

Here we discuss a lot of the essential functionality common to the pandas data
structures. Here’s how to create some of the objects used in the examples from
the previous section:

In [ ]:
index = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index,
                  columns=['A', 'B', 'C'])


<a id='basics-head-tail'></a>

## Head and tail

To view a small sample of a Series or DataFrame object, use the
`head()` and `tail()` methods. The default number
of elements to display is five, but you may pass a custom number.

In [ ]:
long_series = pd.Series(np.random.randn(1000))
long_series.head()
long_series.tail(3)


<a id='basics-attrs'></a>

## Attributes and underlying data

pandas objects have a number of attributes enabling you to access the metadata

- **shape**: gives the axis dimensions of the object, consistent with ndarray  
- 
  <dl style='margin: 20px 0;'>
  <dt>Axis labels</dt>
  <dd>
  - **Series**: *index* (only axis)  
  - **DataFrame**: *index* (rows) and *columns*  
  </dd>
  
  </dl>
  


Note, **these attributes can be safely assigned to**!

In [ ]:
df[:2]
df.columns = [x.lower() for x in df.columns]
df

Pandas objects (`Index`, `Series`, `DataFrame`) can be
thought of as containers for arrays, which hold the actual data and do the
actual computation. For many types, the underlying array is a
[`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray). However, pandas and 3rd party libraries may *extend*
NumPy’s type system to add support for custom arrays
(see [dtypes](#basics-dtypes)).

To get the actual data inside a `Index` or `Series`, use
the `.array` property

In [ ]:
s.array
s.index.array

`array` will always be an `ExtensionArray`.
The exact details of what an `ExtensionArray` is and why pandas uses them is a bit
beyond the scope of this introduction. See [dtypes](#basics-dtypes) for more.

If you know you need a NumPy array, use `to_numpy()`
or `numpy.asarray()`.

In [ ]:
s.to_numpy()
np.asarray(s)

When the Series or Index is backed by
an `ExtensionArray`, `to_numpy()`
may involve copying data and coercing values. See [dtypes](#basics-dtypes) for more.

`to_numpy()` gives some control over the `dtype` of the
resulting [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray). For example, consider datetimes with timezones.
NumPy doesn’t have a dtype to represent timezone-aware datetimes, so there
are two possibly useful representations:

1. An object-dtype [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) with `Timestamp` objects, each
  with the correct `tz`  
1. A `datetime64[ns]` -dtype [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray), where the values have
  been converted to UTC and the timezone discarded  


Timezones may be preserved with `dtype=object`

In [ ]:
ser = pd.Series(pd.date_range('2000', periods=2, tz="CET"))
ser.to_numpy(dtype=object)

Or thrown away with `dtype='datetime64[ns]'`

In [ ]:
ser.to_numpy(dtype="datetime64[ns]")

Getting the “raw data” inside a `DataFrame` is possibly a bit more
complex. When your `DataFrame` only has a single data type for all the
columns, `DataFrame.to_numpy()` will return the underlying data:

In [ ]:
df.to_numpy()

If a DataFrame contains homogeneously-typed data, the ndarray can
actually be modified in-place, and the changes will be reflected in the data
structure. For heterogeneous data (e.g. some of the DataFrame’s columns are not
all the same dtype), this will not be the case. The values attribute itself,
unlike the axis labels, cannot be assigned to.

>**Note**
>
>When working with heterogeneous data, the dtype of the resulting ndarray
will be chosen to accommodate all of the data involved. For example, if
strings are involved, the result will be of object dtype. If there are only
floats and integers, the resulting array will be of float dtype.

In the past, pandas recommended `Series.values` or `DataFrame.values`
for extracting the data from a Series or DataFrame. You’ll still find references
to these in old code bases and online. Going forward, we recommend avoiding
`.values` and using `.array` or `.to_numpy()`. `.values` has the following
drawbacks:

1. When your Series contains an [extension type](development/extending.ipynb#extending-extension-types), it’s
  unclear whether `Series.values` returns a NumPy array or the extension array.
  `Series.array` will always return an `ExtensionArray`, and will never
  copy data. `Series.to_numpy()` will always return a NumPy array,
  potentially at the cost of copying / coercing values.  
1. When your DataFrame contains a mixture of data types, `DataFrame.values` may
  involve copying data and coercing values to a common dtype, a relatively expensive
  operation. `DataFrame.to_numpy()`, being a method, makes it clearer that the
  returned NumPy array may not be a view on the same data in the DataFrame.  



<a id='basics-accelerate'></a>

## Accelerated operations

pandas has support for accelerating certain types of binary numerical and boolean operations using
the `numexpr` library and the `bottleneck` libraries.

These libraries are especially useful when dealing with large data sets, and provide large
speedups. `numexpr` uses smart chunking, caching, and multiple cores. `bottleneck` is
a set of specialized cython routines that are especially fast when dealing with arrays that have
`nans`.

Here is a sample (using 100 column x 100,000 row `DataFrames`):

``````|Operation|0.11.0 (ms)|Prior Version (ms)|Ratio to Prior|
|:-----------------------:|:-----------------------:|:-----------------------:|:-----------------------:|
|df1 > df2|13.32|125.35|0.1063|
|df1 * df2|21.71|36.63|0.5928|
|df1 + df2|22.04|36.50|0.6039|
You are highly encouraged to install both libraries. See the section
[Recommended Dependencies](install.ipynb#install-recommended-dependencies) for more installation info.

These are both enabled to be used by default, you can control this by setting the options:

New in version 0.20.0.

In [ ]:
pd.set_option('compute.use_bottleneck', False)
pd.set_option('compute.use_numexpr', False)


<a id='basics-binop'></a>

## Flexible binary operations

With binary operations between pandas data structures, there are two key points
of interest:

- Broadcasting behavior between higher- (e.g. DataFrame) and
  lower-dimensional (e.g. Series) objects.  
- Missing data in computations.  


We will demonstrate how to manage these issues independently, though they can
be handled simultaneously.

### Matching / broadcasting behavior

DataFrame has the methods `add()`, `sub()`,
`mul()`, `div()` and related functions
`radd()`, `rsub()`, …
for carrying out binary operations. For broadcasting behavior,
Series input is of primary interest. Using these functions, you can use to
either match on the *index* or *columns* via the **axis** keyword:

In [ ]:
df = pd.DataFrame({
    'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
    'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
    'three': pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})
df
row = df.iloc[1]
column = df['two']

df.sub(row, axis='columns')
df.sub(row, axis=1)

df.sub(column, axis='index')
df.sub(column, axis=0)

Furthermore you can align a level of a MultiIndexed DataFrame with a Series.

In [ ]:
dfmi = df.copy()
dfmi.index = pd.MultiIndex.from_tuples([(1, 'a'), (1, 'b'),
                                        (1, 'c'), (2, 'a')],
                                       names=['first', 'second'])
dfmi.sub(column, axis=0, level='second')

Series and Index also support the [`divmod()`](https://docs.python.org/3/library/functions.html#divmod) builtin. This function takes
the floor division and modulo operation at the same time returning a two-tuple
of the same type as the left hand side. For example:

In [ ]:
s = pd.Series(np.arange(10))
s
div, rem = divmod(s, 3)
div
rem

idx = pd.Index(np.arange(10))
idx
div, rem = divmod(idx, 3)
div
rem

We can also do elementwise [`divmod()`](https://docs.python.org/3/library/functions.html#divmod):

In [ ]:
div, rem = divmod(s, [2, 2, 3, 3, 4, 4, 5, 5, 6, 6])
div
rem

### Missing data / operations with fill values

In Series and DataFrame, the arithmetic functions have the option of inputting
a *fill_value*, namely a value to substitute when at most one of the values at
a location are missing. For example, when adding two DataFrame objects, you may
wish to treat NaN as 0 unless both DataFrames are missing that value, in which
case the result will be NaN (you can later replace NaN with some other value
using `fillna` if you wish).

In [ ]:
df
df2
df + df2
df.add(df2, fill_value=0)


<a id='basics-compare'></a>

### Flexible comparisons

Series and DataFrame have the binary comparison methods `eq`, `ne`, `lt`, `gt`,
`le`, and `ge` whose behavior is analogous to the binary
arithmetic operations described above:

In [ ]:
df.gt(df2)
df2.ne(df)

These operations produce a pandas object of the same type as the left-hand-side
input that is of dtype `bool`. These `boolean` objects can be used in
indexing operations, see the section on [Boolean indexing](user_guide/indexing.ipynb#indexing-boolean).


<a id='basics-reductions'></a>

### Boolean reductions

You can apply the reductions: `empty`, `any()`,
`all()`, and `bool()` to provide a
way to summarize a boolean result.

In [ ]:
(df > 0).all()
(df > 0).any()

You can reduce to a final boolean value.

In [ ]:
(df > 0).any().any()

You can test if a pandas object is empty, via the `empty` property.

In [ ]:
df.empty
pd.DataFrame(columns=list('ABC')).empty

To evaluate single-element pandas objects in a boolean context, use the method
`bool()`:

In [ ]:
pd.Series([True]).bool()
pd.Series([False]).bool()
pd.DataFrame([[True]]).bool()
pd.DataFrame([[False]]).bool()

You might be tempted to do the following:

In [ ]:
>>> if df:
...     pass

Or

In [ ]:
>>> df and df2

These will both raise errors, as you are trying to compare multiple values.:

In [ ]:
ValueError: The truth value of an array is ambiguous. Use a.empty, a.any() or a.all().

See [gotchas](user_guide/gotchas.ipynb#gotchas-truth) for a more detailed discussion.


<a id='basics-equals'></a>

### Comparing if objects are equivalent

Often you may find that there is more than one way to compute the same
result.  As a simple example, consider `df + df` and `df * 2`. To test
that these two computations produce the same result, given the tools
shown above, you might imagine using `(df + df == df * 2).all()`. But in
fact, this expression is False:

In [ ]:
df + df == df * 2
(df + df == df * 2).all()

Notice that the boolean DataFrame `df + df == df * 2` contains some False values!
This is because NaNs do not compare as equals:

In [ ]:
np.nan == np.nan

So, NDFrames (such as Series and DataFrames)
have an `equals()` method for testing equality, with NaNs in
corresponding locations treated as equal.

In [ ]:
(df + df).equals(df * 2)

Note that the Series or DataFrame index needs to be in the same order for
equality to be True:

In [ ]:
df1 = pd.DataFrame({'col': ['foo', 0, np.nan]})
df2 = pd.DataFrame({'col': [np.nan, 0, 'foo']}, index=[2, 1, 0])
df1.equals(df2)
df1.equals(df2.sort_index())

### Comparing array-like objects

You can conveniently perform element-wise comparisons when comparing a pandas
data structure with a scalar value:

In [ ]:
pd.Series(['foo', 'bar', 'baz']) == 'foo'
pd.Index(['foo', 'bar', 'baz']) == 'foo'

Pandas also handles element-wise comparisons between different array-like
objects of the same length:

In [ ]:
pd.Series(['foo', 'bar', 'baz']) == pd.Index(['foo', 'bar', 'qux'])
pd.Series(['foo', 'bar', 'baz']) == np.array(['foo', 'bar', 'qux'])

Trying to compare `Index` or `Series` objects of different lengths will
raise a ValueError:

```ipython
In [55]: pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar'])
ValueError: Series lengths must match to compare

In [56]: pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo'])
ValueError: Series lengths must match to compare
```


Note that this is different from the NumPy behavior where a comparison can
be broadcast:

In [ ]:
np.array([1, 2, 3]) == np.array([2])

or it can return False if broadcasting can not be done:

### Combining overlapping data sets

A problem occasionally arising is the combination of two similar data sets
where values in one are preferred over the other. An example would be two data
series representing a particular economic indicator where one is considered to
be of “higher quality”. However, the lower quality series might extend further
back in history or have more complete data coverage. As such, we would like to
combine two DataFrame objects where missing values in one DataFrame are
conditionally filled with like-labeled values from the other DataFrame. The
function implementing this operation is `combine_first()`,
which we illustrate:

In [ ]:
df1 = pd.DataFrame({'A': [1., np.nan, 3., 5., np.nan],
                    'B': [np.nan, 2., 3., np.nan, 6.]})
df2 = pd.DataFrame({'A': [5., 2., 4., np.nan, 3., 7.],
                    'B': [np.nan, np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

### General DataFrame combine

The `combine_first()` method above calls the more general
`DataFrame.combine()`. This method takes another DataFrame
and a combiner function, aligns the input DataFrame and then passes the combiner
function pairs of Series (i.e., columns whose names are the same).

So, for instance, to reproduce `combine_first()` as above:

In [ ]:
def combiner(x, y):
    return np.where(pd.isna(x), y, x)
df1.combine(df2, combiner)


<a id='basics-stats'></a>

## Descriptive statistics

There exists a large number of methods for computing descriptive statistics and
other related operations on Series, DataFrame. Most of these
are aggregations (hence producing a lower-dimensional result) like
`sum()`, `mean()`, and `quantile()`,
but some of them, like `cumsum()` and `cumprod()`,
produce an object of the same size. Generally speaking, these methods take an
**axis** argument, just like *ndarray.{sum, std, …}*, but the axis can be
specified by name or integer:

- **Series**: no axis argument needed  
- **DataFrame**: “index” (axis=0, default), “columns” (axis=1)  


For example:

In [ ]:
df
df.mean(0)
df.mean(1)

All such methods have a `skipna` option signaling whether to exclude missing
data (`True` by default):

In [ ]:
df.sum(0, skipna=False)
df.sum(axis=1, skipna=True)

Combined with the broadcasting / arithmetic behavior, one can describe various
statistical procedures, like standardization (rendering data zero mean and
standard deviation 1), very concisely:

In [ ]:
ts_stand = (df - df.mean()) / df.std()
ts_stand.std()
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
xs_stand.std(1)

Note that methods like `cumsum()` and `cumprod()`
preserve the location of `NaN` values. This is somewhat different from
`expanding()` and `rolling()`.
For more details please see [this note](user_guide/computation.ipynb#stats-moments-expanding-note).

In [ ]:
df.cumsum()

Here is a quick reference summary table of common functions. Each also takes an
optional `level` parameter which applies only if the object has a
[hierarchical index](user_guide/advanced.ipynb#advanced-hierarchical).

````````````````````````````````````````|Function|Description|
|:------------------:|:------------------------------------------------------------------------------:|
|count|Number of non-NA observations|
|sum|Sum of values|
|mean|Mean of values|
|mad|Mean absolute deviation|
|median|Arithmetic median of values|
|min|Minimum|
|max|Maximum|
|mode|Mode|
|abs|Absolute Value|
|prod|Product of values|
|std|Bessel-corrected sample standard deviation|
|var|Unbiased variance|
|sem|Standard error of the mean|
|skew|Sample skewness (3rd moment)|
|kurt|Sample kurtosis (4th moment)|
|quantile|Sample quantile (value at %)|
|cumsum|Cumulative sum|
|cumprod|Cumulative product|
|cummax|Cumulative maximum|
|cummin|Cumulative minimum|
Note that by chance some NumPy methods, like `mean`, `std`, and `sum`,
will exclude NAs on Series input by default:

In [ ]:
np.mean(df['one'])
np.mean(df['one'].to_numpy())

`Series.nunique()` will return the number of unique non-NA values in a
Series:

In [ ]:
series = pd.Series(np.random.randn(500))
series[20:500] = np.nan
series[10:20] = 5
series.nunique()


<a id='basics-describe'></a>

### Summarizing data: describe

There is a convenient `describe()` function which computes a variety of summary
statistics about a Series or the columns of a DataFrame (excluding NAs of
course):

In [ ]:
series = pd.Series(np.random.randn(1000))
series[::2] = np.nan
series.describe()
frame = pd.DataFrame(np.random.randn(1000, 5),
                     columns=['a', 'b', 'c', 'd', 'e'])
frame.iloc[::2] = np.nan
frame.describe()

You can select specific percentiles to include in the output:

In [ ]:
series.describe(percentiles=[.05, .25, .75, .95])

By default, the median is always included.

For a non-numerical Series object, `describe()` will give a simple
summary of the number of unique values and most frequently occurring values:

In [ ]:
s = pd.Series(['a', 'a', 'b', 'b', 'a', 'a', np.nan, 'c', 'd', 'a'])
s.describe()

Note that on a mixed-type DataFrame object, `describe()` will
restrict the summary to include only numerical columns or, if none are, only
categorical columns:

In [ ]:
frame = pd.DataFrame({'a': ['Yes', 'Yes', 'No', 'No'], 'b': range(4)})
frame.describe()

This behavior can be controlled by providing a list of types as `include`/`exclude`
arguments. The special value `all` can also be used:

In [ ]:
frame.describe(include=['object'])
frame.describe(include=['number'])
frame.describe(include='all')

That feature relies on [select_dtypes](#basics-selectdtypes). Refer to
there for details about accepted inputs.


<a id='basics-idxmin'></a>

### Index of min/max values

The `idxmin()` and `idxmax()` functions on Series
and DataFrame compute the index labels with the minimum and maximum
corresponding values:

In [ ]:
s1 = pd.Series(np.random.randn(5))
s1
s1.idxmin(), s1.idxmax()

df1 = pd.DataFrame(np.random.randn(5, 3), columns=['A', 'B', 'C'])
df1
df1.idxmin(axis=0)
df1.idxmax(axis=1)

When there are multiple rows (or columns) matching the minimum or maximum
value, `idxmin()` and `idxmax()` return the first
matching index:

In [ ]:
df3 = pd.DataFrame([2, 1, 1, 3, np.nan], columns=['A'], index=list('edcba'))
df3
df3['A'].idxmin()

>**Note**
>
>`idxmin` and `idxmax` are called `argmin` and `argmax` in NumPy.


<a id='basics-discretization'></a>

### Value counts (histogramming) / mode

The `value_counts()` Series method and top-level function computes a histogram
of a 1D array of values. It can also be used as a function on regular arrays:

In [ ]:
data = np.random.randint(0, 7, size=50)
data
s = pd.Series(data)
s.value_counts()
pd.value_counts(data)

Similarly, you can get the most frequently occurring value(s) (the mode) of the values in a Series or DataFrame:

In [ ]:
s5 = pd.Series([1, 1, 3, 3, 3, 5, 5, 7, 7, 7])
s5.mode()
df5 = pd.DataFrame({"A": np.random.randint(0, 7, size=50),
                    "B": np.random.randint(-10, 15, size=50)})
df5.mode()

### Discretization and quantiling

Continuous values can be discretized using the `cut()` (bins based on values)
and `qcut()` (bins based on sample quantiles) functions:

In [ ]:
arr = np.random.randn(20)
factor = pd.cut(arr, 4)
factor

factor = pd.cut(arr, [-5, -1, 0, 1, 5])
factor

`qcut()` computes sample quantiles. For example, we could slice up some
normally distributed data into equal-size quartiles like so:

In [ ]:
arr = np.random.randn(30)
factor = pd.qcut(arr, [0, .25, .5, .75, 1])
factor
pd.value_counts(factor)

We can also pass infinite values to define the bins:

In [ ]:
arr = np.random.randn(20)
factor = pd.cut(arr, [-np.inf, 0, np.inf])
factor


<a id='basics-apply'></a>

## Function application

To apply your own or another library’s functions to pandas objects,
you should be aware of the three methods below. The appropriate
method to use depends on whether your function expects to operate
on an entire `DataFrame` or `Series`, row- or column-wise, or elementwise.

1. [Tablewise Function Application](#tablewise-function-application): `pipe()`  
1. [Row or Column-wise Function Application](#row-or-column-wise-function-application): `apply()`  
1. [Aggregation API](#aggregation-api): `agg()` and `transform()`  
1. [Applying Elementwise Functions](#applying-elementwise-functions): `applymap()`  



<a id='basics-pipe'></a>

### Tablewise function application

`DataFrames` and `Series` can of course just be passed into functions.
However, if the function needs to be called in a chain, consider using the `pipe()` method.
Compare the following

In [ ]:
# f, g, and h are functions taking and returning ``DataFrames``
>>> f(g(h(df), arg1=1), arg2=2, arg3=3)

with the equivalent

In [ ]:
>>> (df.pipe(h)
...    .pipe(g, arg1=1)
...    .pipe(f, arg2=2, arg3=3))

Pandas encourages the second style, which is known as method chaining.
`pipe` makes it easy to use your own or another library’s functions
in method chains, alongside pandas’ methods.

In the example above, the functions `f`, `g`, and `h` each expected the `DataFrame` as the first positional argument.
What if the function you wish to apply takes its data as, say, the second argument?
In this case, provide `pipe` with a tuple of `(callable, data_keyword)`.
`.pipe` will route the `DataFrame` to the argument specified in the tuple.

For example, we can fit a regression using statsmodels. Their API expects a formula first and a `DataFrame` as the second argument, `data`. We pass in the function, keyword pair `(sm.ols, 'data')` to `pipe`:

The pipe method is inspired by unix pipes and more recently [dplyr](https://github.com/hadley/dplyr) and [magrittr](https://github.com/smbache/magrittr), which
have introduced the popular `(%>%)` (read pipe) operator for [R](https://www.r-project.org).
The implementation of `pipe` here is quite clean and feels right at home in python.
We encourage you to view the source code of `pipe()`.

### Row or column-wise function application

Arbitrary functions can be applied along the axes of a DataFrame
using the `apply()` method, which, like the descriptive
statistics methods, takes an optional `axis` argument:

In [ ]:
df.apply(np.mean)
df.apply(np.mean, axis=1)
df.apply(lambda x: x.max() - x.min())
df.apply(np.cumsum)
df.apply(np.exp)

The `apply()` method will also dispatch on a string method name.

In [ ]:
df.apply('mean')
df.apply('mean', axis=1)

The return type of the function passed to `apply()` affects the
type of the final output from `DataFrame.apply` for the default behaviour:

- If the applied function returns a `Series`, the final output is a `DataFrame`.
  The columns match the index of the `Series` returned by the applied function.  
- If the applied function returns any other type, the final output is a `Series`.  


This default behaviour can be overridden using the `result_type`, which
accepts three options: `reduce`, `broadcast`, and `expand`.
These will determine how list-likes return values expand (or not) to a `DataFrame`.

`apply()` combined with some cleverness can be used to answer many questions
about a data set. For example, suppose we wanted to extract the date where the
maximum value for each column occurred:

In [ ]:
tsdf = pd.DataFrame(np.random.randn(1000, 3), columns=['A', 'B', 'C'],
                    index=pd.date_range('1/1/2000', periods=1000))
tsdf.apply(lambda x: x.idxmax())

You may also pass additional arguments and keyword arguments to the `apply()`
method. For instance, consider the following function you would like to apply:

In [ ]:
def subtract_and_divide(x, sub, divide=1):
    return (x - sub) / divide

You may then apply this function as follows:

In [ ]:
df.apply(subtract_and_divide, args=(5,), divide=3)

Another useful feature is the ability to pass Series methods to carry out some
Series operation on each column or row:

In [ ]:
tsdf
tsdf.apply(pd.Series.interpolate)

Finally, `apply()` takes an argument `raw` which is False by default, which
converts each row or column into a Series before applying the function. When
set to True, the passed function will instead receive an ndarray object, which
has positive performance implications if you do not need the indexing
functionality.


<a id='basics-aggregate'></a>

### Aggregation API

New in version 0.20.0.

The aggregation API allows one to express possibly multiple aggregation operations in a single concise way.
This API is similar across pandas objects, see [groupby API](user_guide/groupby.ipynb#groupby-aggregate), the
[window functions API](user_guide/computation.ipynb#stats-aggregate), and the [resample API](user_guide/timeseries.ipynb#timeseries-aggregate).
The entry point for aggregation is `DataFrame.aggregate()`, or the alias
`DataFrame.agg()`.

We will use a similar starting frame from above:

In [ ]:
tsdf = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'],
                    index=pd.date_range('1/1/2000', periods=10))
tsdf.iloc[3:7] = np.nan
tsdf

Using a single function is equivalent to `apply()`. You can also
pass named methods as strings. These will return a `Series` of the aggregated
output:

In [ ]:
tsdf.agg(np.sum)

tsdf.agg('sum')

# these are equivalent to a ``.sum()`` because we are aggregating
# on a single function
tsdf.sum()

Single aggregations on a `Series` this will return a scalar value:

In [ ]:
tsdf.A.agg('sum')

#### Aggregating with multiple functions

You can pass multiple aggregation arguments as a list.
The results of each of the passed functions will be a row in the resulting `DataFrame`.
These are naturally named from the aggregation function.

In [ ]:
tsdf.agg(['sum'])

Multiple functions yield multiple rows:

In [ ]:
tsdf.agg(['sum', 'mean'])

On a `Series`, multiple functions return a `Series`, indexed by the function names:

In [ ]:
tsdf.A.agg(['sum', 'mean'])

Passing a `lambda` function will yield a `<lambda>` named row:

In [ ]:
tsdf.A.agg(['sum', lambda x: x.mean()])

Passing a named function will yield that name for the row:

In [ ]:
def mymean(x):
    return x.mean()

tsdf.A.agg(['sum', mymean])

#### Aggregating with a dict

Passing a dictionary of column names to a scalar or a list of scalars, to `DataFrame.agg`
allows you to customize which functions are applied to which columns. Note that the results
are not in any particular order, you can use an `OrderedDict` instead to guarantee ordering.

In [ ]:
tsdf.agg({'A': 'mean', 'B': 'sum'})

Passing a list-like will generate a `DataFrame` output. You will get a matrix-like output
of all of the aggregators. The output will consist of all unique functions. Those that are
not noted for a particular column will be `NaN`:

In [ ]:
tsdf.agg({'A': ['mean', 'min'], 'B': 'sum'})


<a id='basics-aggregation-mixed-dtypes'></a>

#### Mixed dtypes

When presented with mixed dtypes that cannot aggregate, `.agg` will only take the valid
aggregations. This is similar to how groupby `.agg` works.

In [ ]:
mdf = pd.DataFrame({'A': [1, 2, 3],
                    'B': [1., 2., 3.],
                    'C': ['foo', 'bar', 'baz'],
                    'D': pd.date_range('20130101', periods=3)})
mdf.dtypes

In [ ]:
mdf.agg(['min', 'sum'])


<a id='basics-aggregation-custom-describe'></a>

#### Custom describe

With `.agg()` is it possible to easily create a custom describe function, similar
to the built in [describe function](#basics-describe).

In [ ]:
from functools import partial

q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = '25%'
q_75 = partial(pd.Series.quantile, q=0.75)
q_75.__name__ = '75%'

tsdf.agg(['count', 'mean', 'std', 'min', q_25, 'median', q_75, 'max'])


<a id='basics-transform'></a>

### Transform API

New in version 0.20.0.

The `transform()` method returns an object that is indexed the same (same size)
as the original. This API allows you to provide *multiple* operations at the same
time rather than one-by-one. Its API is quite similar to the `.agg` API.

We create a frame similar to the one used in the above sections.

In [ ]:
tsdf = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'],
                    index=pd.date_range('1/1/2000', periods=10))
tsdf.iloc[3:7] = np.nan
tsdf

Transform the entire frame. `.transform()` allows input functions as: a NumPy function, a string
function name or a user defined function.

Here `transform()` received a single function; this is equivalent to a ufunc application.

In [ ]:
np.abs(tsdf)

Passing a single function to `.transform()` with a `Series` will yield a single `Series` in return.

In [ ]:
tsdf.A.transform(np.abs)

#### Transform with multiple functions

Passing multiple functions will yield a column MultiIndexed DataFrame.
The first level will be the original frame column names; the second level
will be the names of the transforming functions.

In [ ]:
tsdf.transform([np.abs, lambda x: x + 1])

Passing multiple functions to a Series will yield a DataFrame. The
resulting column names will be the transforming functions.

In [ ]:
tsdf.A.transform([np.abs, lambda x: x + 1])

#### Transforming with a dict

Passing a dict of functions will allow selective transforming per column.

In [ ]:
tsdf.transform({'A': np.abs, 'B': lambda x: x + 1})

Passing a dict of lists will generate a MultiIndexed DataFrame with these
selective transforms.


<a id='basics-elementwise'></a>

### Applying elementwise functions

Since not all functions can be vectorized (accept NumPy arrays and return
another array or value), the methods `applymap()` on DataFrame
and analogously `map()` on Series accept any Python function taking
a single value and returning a single value. For example:

In [ ]:
df4

def f(x):
    return len(str(x))

df4['one'].map(f)
df4.applymap(f)

`Series.map()` has an additional feature; it can be used to easily
“link” or “map” values defined by a secondary series. This is closely related
to [merging/joining functionality](user_guide/merging.ipynb#merging):

In [ ]:
s = pd.Series(['six', 'seven', 'six', 'seven', 'six'],
              index=['a', 'b', 'c', 'd', 'e'])
t = pd.Series({'six': 6., 'seven': 7.})
s
s.map(t)


<a id='basics-reindexing'></a>

## Reindexing and altering labels

`reindex()` is the fundamental data alignment method in pandas.
It is used to implement nearly all other features relying on label-alignment
functionality. To *reindex* means to conform the data to match a given set of
labels along a particular axis. This accomplishes several things:

- Reorders the existing data to match a new set of labels  
- Inserts missing value (NA) markers in label locations where no data for
  that label existed  
- If specified, **fill** data for missing labels using logic (highly relevant
  to working with time series data)  


Here is a simple example:

In [ ]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
s
s.reindex(['e', 'b', 'f', 'd'])

Here, the `f` label was not contained in the Series and hence appears as
`NaN` in the result.

With a DataFrame, you can simultaneously reindex the index and columns:

In [ ]:
df
df.reindex(index=['c', 'f', 'b'], columns=['three', 'two', 'one'])

You may also use `reindex` with an `axis` keyword:

In [ ]:
df.reindex(['c', 'f', 'b'], axis='index')

Note that the `Index` objects containing the actual axis labels can be
**shared** between objects. So if we have a Series and a DataFrame, the
following can be done:

In [ ]:
rs = s.reindex(df.index)
rs
rs.index is df.index

This means that the reindexed Series’s index is the same Python object as the
DataFrame’s index.

New in version 0.21.0.

`DataFrame.reindex()` also supports an “axis-style” calling convention,
where you specify a single `labels` argument and the `axis` it applies to.

In [ ]:
df.reindex(['c', 'f', 'b'], axis='index')
df.reindex(['three', 'two', 'one'], axis='columns')

[MultiIndex / Advanced Indexing](user_guide/advanced.ipynb#advanced) is an even more concise way of
doing reindexing.

>**Note**
>
>When writing performance-sensitive code, there is a good reason to spend
some time becoming a reindexing ninja: **many operations are faster on
pre-aligned data**. Adding two unaligned DataFrames internally triggers a
reindexing step. For exploratory analysis you will hardly notice the
difference (because `reindex` has been heavily optimized), but when CPU
cycles matter sprinkling a few explicit `reindex` calls here and there can
have an impact.


<a id='basics-reindex-like'></a>

### Reindexing to align with another object

You may wish to take an object and reindex its axes to be labeled the same as
another object. While the syntax for this is straightforward albeit verbose, it
is a common enough operation that the `reindex_like()` method is
available to make this simpler:

In [ ]:
df2
df3
df.reindex_like(df2)


<a id='basics-align'></a>

### Aligning objects with each other with `align`

The `align()` method is the fastest way to simultaneously align two objects. It
supports a `join` argument (related to [joining and merging](user_guide/merging.ipynb#merging)):

> - `join='outer'`: take the union of the indexes (default)  
- `join='left'`: use the calling object’s index  
- `join='right'`: use the passed object’s index  
- `join='inner'`: intersect the indexes  



It returns a tuple with both of the reindexed Series:

In [ ]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
s1 = s[:4]
s2 = s[1:]
s1.align(s2)
s1.align(s2, join='inner')
s1.align(s2, join='left')


<a id='basics-df-join'></a>
For DataFrames, the join method will be applied to both the index and the
columns by default:

In [ ]:
df.align(df2, join='inner')

You can also pass an `axis` option to only align on the specified axis:

In [ ]:
df.align(df2, join='inner', axis=0)


<a id='basics-align-frame-series'></a>
If you pass a Series to `DataFrame.align()`, you can choose to align both
objects either on the DataFrame’s index or columns using the `axis` argument:

In [ ]:
df.align(df2.iloc[0], axis=1)


<a id='basics-reindex-fill'></a>

### Filling while reindexing

`reindex()` takes an optional parameter `method` which is a
filling method chosen from the following table:

|Method|Action|
|:----------------------------:|:------------------------------------------------:|
|pad / ffill|Fill values forward|
|bfill / backfill|Fill values backward|
|nearest|Fill from the nearest index value|
We illustrate these fill methods on a simple Series:

In [ ]:
rng = pd.date_range('1/3/2000', periods=8)
ts = pd.Series(np.random.randn(8), index=rng)
ts2 = ts[[0, 3, 6]]
ts
ts2

ts2.reindex(ts.index)
ts2.reindex(ts.index, method='ffill')
ts2.reindex(ts.index, method='bfill')
ts2.reindex(ts.index, method='nearest')

These methods require that the indexes are **ordered** increasing or
decreasing.

Note that the same result could have been achieved using
[fillna](user_guide/missing_data.ipynb#missing-data-fillna) (except for `method='nearest'`) or
[interpolate](user_guide/missing_data.ipynb#missing-data-interpolate):

In [ ]:
ts2.reindex(ts.index).fillna(method='ffill')

`reindex()` will raise a ValueError if the index is not monotonically
increasing or decreasing. `fillna()` and `interpolate()`
will not perform any checks on the order of the index.


<a id='basics-limits-on-reindex-fill'></a>

### Limits on filling while reindexing

The `limit` and `tolerance` arguments provide additional control over
filling while reindexing. Limit specifies the maximum count of consecutive
matches:

In [ ]:
ts2.reindex(ts.index, method='ffill', limit=1)

In contrast, tolerance specifies the maximum distance between the index and
indexer values:

In [ ]:
ts2.reindex(ts.index, method='ffill', tolerance='1 day')

Notice that when used on a `DatetimeIndex`, `TimedeltaIndex` or
`PeriodIndex`, `tolerance` will coerced into a `Timedelta` if possible.
This allows you to specify tolerance with appropriate strings.


<a id='basics-drop'></a>

### Dropping labels from an axis

A method closely related to `reindex` is the `drop()` function.
It removes a set of labels from an axis:

In [ ]:
df
df.drop(['a', 'd'], axis=0)
df.drop(['one'], axis=1)

Note that the following also works, but is a bit less obvious / clean:

In [ ]:
df.reindex(df.index.difference(['a', 'd']))


<a id='basics-rename'></a>

### Renaming / mapping labels

The `rename()` method allows you to relabel an axis based on some
mapping (a dict or Series) or an arbitrary function.

In [ ]:
s
s.rename(str.upper)

If you pass a function, it must return a value when called with any of the
labels (and must produce a set of unique values). A dict or
Series can also be used:

In [ ]:
df.rename(columns={'one': 'foo', 'two': 'bar'},
          index={'a': 'apple', 'b': 'banana', 'd': 'durian'})

If the mapping doesn’t include a column/index label, it isn’t renamed. Note that
extra labels in the mapping don’t throw an error.

New in version 0.21.0.

`DataFrame.rename()` also supports an “axis-style” calling convention, where
you specify a single `mapper` and the `axis` to apply that mapping to.

In [ ]:
df.rename({'one': 'foo', 'two': 'bar'}, axis='columns')
df.rename({'a': 'apple', 'b': 'banana', 'd': 'durian'}, axis='index')

The `rename()` method also provides an `inplace` named
parameter that is by default `False` and copies the underlying data. Pass
`inplace=True` to rename the data in place.

New in version 0.18.0.

Finally, `rename()` also accepts a scalar or list-like
for altering the `Series.name` attribute.

In [ ]:
s.rename("scalar-name")


<a id='basics-rename-axis'></a>
New in version 0.24.0.

The methods `rename_axis()` and `rename_axis()`
allow specific names of a MultiIndex to be changed (as opposed to the
labels).

In [ ]:
df = pd.DataFrame({'x': [1, 2, 3, 4, 5, 6],
                   'y': [10, 20, 30, 40, 50, 60]},
                  index=pd.MultiIndex.from_product([['a', 'b', 'c'], [1, 2]],
                  names=['let', 'num']))
df
df.rename_axis(index={'let': 'abc'})
df.rename_axis(index=str.upper)


<a id='basics-iteration'></a>

## Iteration

The behavior of basic iteration over pandas objects depends on the type.
When iterating over a Series, it is regarded as array-like, and basic iteration
produces the values. DataFrames follow the dict-like convention of iterating
over the “keys” of the objects.

In short, basic iteration (`for i in object`) produces:

- **Series**: values  
- **DataFrame**: column labels  


Thus, for example, iterating over a DataFrame gives you the column names:

In [ ]:
df = pd.DataFrame({'col1': np.random.randn(3),
                   'col2': np.random.randn(3)}, index=['a', 'b', 'c'])

for col in df:
    print(col)

Pandas objects also have the dict-like `items()` method to
iterate over the (key, value) pairs.

To iterate over the rows of a DataFrame, you can use the following methods:

- `iterrows()`: Iterate over the rows of a DataFrame as (index, Series) pairs.
  This converts the rows to Series objects, which can change the dtypes and has some
  performance implications.  
- `itertuples()`: Iterate over the rows of a DataFrame
  as namedtuples of the values.  This is a lot faster than
  `iterrows()`, and is in most cases preferable to use
  to iterate over the values of a DataFrame.  


Iterating through pandas objects is generally **slow**. In many cases,
iterating manually over the rows is not needed and can be avoided with
one of the following approaches:

- Look for a *vectorized* solution: many operations can be performed using
  built-in methods or NumPy functions, (boolean) indexing, …  
- When you have a function that cannot work on the full DataFrame/Series
  at once, it is better to use `apply()` instead of iterating
  over the values. See the docs on [function application](#basics-apply).  
- If you need to do iterative manipulations on the values but performance is
  important, consider writing the inner loop with cython or numba.
  See the [enhancing performance](user_guide/enhancingperf.ipynb#enhancingperf) section for some
  examples of this approach.  


You should **never modify** something you are iterating over.
This is not guaranteed to work in all cases. Depending on the
data types, the iterator returns a copy and not a view, and writing
to it will have no effect!

For example, in the following case setting the value has no effect:

In [ ]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': ['a', 'b', 'c']})

for index, row in df.iterrows():
    row['a'] = 10

df

### items

Consistent with the dict-like interface, `items()` iterates
through key-value pairs:

- **Series**: (index, scalar value) pairs  
- **DataFrame**: (column, Series) pairs  


For example:

In [ ]:
for label, ser in df.items():
    print(label)
    print(ser)


<a id='basics-iterrows'></a>

### iterrows

`iterrows()` allows you to iterate through the rows of a
DataFrame as Series objects. It returns an iterator yielding each
index value along with a Series containing the data in each row:

In [ ]:
for row_index, row in df.iterrows():
    print(row_index, row, sep='\n')

>**Note**
>
>Because `iterrows()` returns a Series for each row,
it does **not** preserve dtypes across the rows (dtypes are
preserved across columns for DataFrames). For example,

In [ ]:
df_orig = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])
df_orig.dtypes
row = next(df_orig.iterrows())[1]
row

All values in `row`, returned as a Series, are now upcasted
to floats, also the original integer value in column x:

In [ ]:
row['int'].dtype
df_orig['int'].dtype

To preserve dtypes while iterating over the rows, it is better
to use `itertuples()` which returns namedtuples of the values
and which is generally much faster than `iterrows()`.

For instance, a contrived way to transpose the DataFrame would be:

In [ ]:
df2 = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
print(df2)
print(df2.T)

df2_t = pd.DataFrame({idx: values for idx, values in df2.iterrows()})
print(df2_t)

### itertuples

The `itertuples()` method will return an iterator
yielding a namedtuple for each row in the DataFrame. The first element
of the tuple will be the row’s corresponding index value, while the
remaining values are the row values.

For instance:

In [ ]:
for row in df.itertuples():
    print(row)

This method does not convert the row to a Series object; it merely
returns the values inside a namedtuple. Therefore,
`itertuples()` preserves the data type of the values
and is generally faster as `iterrows()`.

>**Note**
>
>The column names will be renamed to positional names if they are
invalid Python identifiers, repeated, or start with an underscore.
With a large number of columns (>255), regular tuples are returned.


<a id='basics-dt-accessors'></a>

## .dt accessor

`Series` has an accessor to succinctly return datetime like properties for the
*values* of the Series, if it is a datetime/period like Series.
This will return a Series, indexed like the existing Series.

In [ ]:
# datetime
s = pd.Series(pd.date_range('20130101 09:10:12', periods=4))
s
s.dt.hour
s.dt.second
s.dt.day

This enables nice expressions like this:

In [ ]:
s[s.dt.day == 2]

You can easily produces tz aware transformations:

In [ ]:
stz = s.dt.tz_localize('US/Eastern')
stz
stz.dt.tz

You can also chain these types of operations:

In [ ]:
s.dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

You can also format datetime values as strings with `Series.dt.strftime()` which
supports the same format as the standard [`strftime()`](https://docs.python.org/3/library/datetime.html#datetime.datetime.strftime).

In [ ]:
# DatetimeIndex
s = pd.Series(pd.date_range('20130101', periods=4))
s
s.dt.strftime('%Y/%m/%d')

In [ ]:
# PeriodIndex
s = pd.Series(pd.period_range('20130101', periods=4))
s
s.dt.strftime('%Y/%m/%d')

The `.dt` accessor works for period and timedelta dtypes.

In [ ]:
# period
s = pd.Series(pd.period_range('20130101', periods=4, freq='D'))
s
s.dt.year
s.dt.day

In [ ]:
# timedelta
s = pd.Series(pd.timedelta_range('1 day 00:00:05', periods=4, freq='s'))
s
s.dt.days
s.dt.seconds
s.dt.components

>**Note**
>
>`Series.dt` will raise a `TypeError` if you access with a non-datetime-like values.

## Vectorized string methods

Series is equipped with a set of string processing methods that make it easy to
operate on each element of the array. Perhaps most importantly, these methods
exclude missing/NA values automatically. These are accessed via the Series’s
`str` attribute and generally have names matching the equivalent (scalar)
built-in string methods. For example:

> 

In [ ]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])
s.str.lower()


Powerful pattern-matching methods are provided as well, but note that
pattern-matching generally uses [regular expressions](https://docs.python.org/3/library/re.html) by default (and in some cases
always uses them).

Please see [Vectorized String Methods](user_guide/text.ipynb#text-string-methods) for a complete
description.


<a id='basics-sorting'></a>

## Sorting

Pandas supports three kinds of sorting: sorting by index labels,
sorting by column values, and sorting by a combination of both.


<a id='basics-sort-index'></a>

### By index

The `Series.sort_index()` and `DataFrame.sort_index()` methods are
used to sort a pandas object by its index levels.

In [ ]:
df = pd.DataFrame({
    'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
    'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
    'three': pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

unsorted_df = df.reindex(index=['a', 'd', 'c', 'b'],
                         columns=['three', 'two', 'one'])
unsorted_df

# DataFrame
unsorted_df.sort_index()
unsorted_df.sort_index(ascending=False)
unsorted_df.sort_index(axis=1)

# Series
unsorted_df['three'].sort_index()


<a id='basics-sort-values'></a>

### By values

The `Series.sort_values()` method is used to sort a Series by its values. The
`DataFrame.sort_values()` method is used to sort a DataFrame by its column or row values.
The optional `by` parameter to `DataFrame.sort_values()` may used to specify one or more columns
to use to determine the sorted order.

In [ ]:
df1 = pd.DataFrame({'one': [2, 1, 1, 1],
                    'two': [1, 3, 2, 4],
                    'three': [5, 4, 3, 2]})
df1.sort_values(by='two')

The `by` parameter can take a list of column names, e.g.:

In [ ]:
df1[['one', 'two', 'three']].sort_values(by=['one', 'two'])

These methods have special treatment of NA values via the `na_position`
argument:

In [ ]:
s[2] = np.nan
s.sort_values()
s.sort_values(na_position='first')


<a id='basics-sort-indexes-and-values'></a>

### By indexes and values

New in version 0.23.0.

Strings passed as the `by` parameter to `DataFrame.sort_values()` may
refer to either columns or index level names.

In [ ]:
# Build MultiIndex
idx = pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('a', 2),
                                ('b', 2), ('b', 1), ('b', 1)])
idx.names = ['first', 'second']

# Build DataFrame
df_multi = pd.DataFrame({'A': np.arange(6, 0, -1)},
                        index=idx)
df_multi

Sort by ‘second’ (index) and ‘A’ (column)

In [ ]:
df_multi.sort_values(by=['second', 'A'])

>**Note**
>
>If a string matches both a column name and an index level name then a
warning is issued and the column takes precedence. This will result in an
ambiguity error in a future version.


<a id='basics-searchsorted'></a>

### searchsorted

Series has the `searchsorted()` method, which works similarly to
[`numpy.ndarray.searchsorted()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.searchsorted.html#numpy.ndarray.searchsorted).

In [ ]:
ser = pd.Series([1, 2, 3])
ser.searchsorted([0, 3])
ser.searchsorted([0, 4])
ser.searchsorted([1, 3], side='right')
ser.searchsorted([1, 3], side='left')
ser = pd.Series([3, 1, 2])
ser.searchsorted([0, 3], sorter=np.argsort(ser))


<a id='basics-nsorted'></a>

### smallest / largest values

`Series` has the `nsmallest()` and `nlargest()` methods which return the
smallest or largest $ n $ values. For a large `Series` this can be much
faster than sorting the entire Series and calling `head(n)` on the result.

In [ ]:
s = pd.Series(np.random.permutation(10))
s
s.sort_values()
s.nsmallest(3)
s.nlargest(3)

`DataFrame` also has the `nlargest` and `nsmallest` methods.

In [ ]:
df = pd.DataFrame({'a': [-2, -1, 1, 10, 8, 11, -1],
                   'b': list('abdceff'),
                   'c': [1.0, 2.0, 4.0, 3.2, np.nan, 3.0, 4.0]})
df.nlargest(3, 'a')
df.nlargest(5, ['a', 'c'])
df.nsmallest(3, 'a')
df.nsmallest(5, ['a', 'c'])


<a id='basics-multiindex-sorting'></a>

### Sorting by a MultiIndex column

You must be explicit about sorting when the column is a MultiIndex, and fully specify
all levels to `by`.

In [ ]:
df1.columns = pd.MultiIndex.from_tuples([('a', 'one'),
                                         ('a', 'two'),
                                         ('b', 'three')])
df1.sort_values(by=('a', 'two'))

## Copying

The `copy()` method on pandas objects copies the underlying data (though not
the axis indexes, since they are immutable) and returns a new object. Note that
**it is seldom necessary to copy objects**. For example, there are only a
handful of ways to alter a DataFrame *in-place*:

- Inserting, deleting, or modifying a column.  
- Assigning to the `index` or `columns` attributes.  
- For homogeneous data, directly modifying the values via the `values`
  attribute or advanced indexing.  


To be clear, no pandas method has the side effect of modifying your data;
almost every method returns a new object, leaving the original object
untouched. If the data is modified, it is because you did so explicitly.


<a id='basics-dtypes'></a>

## dtypes

For the most part, pandas uses NumPy arrays and dtypes for Series or individual
columns of a DataFrame. NumPy provides support for `float`,
`int`, `bool`, `timedelta64[ns]` and `datetime64[ns]` (note that NumPy
does not support timezone-aware datetimes).

Pandas and third-party libraries *extend* NumPy’s type system in a few places.
This section describes the extensions pandas has made internally.
See [Extension types](development/extending.ipynb#extending-extension-types) for how to write your own extension that
works with pandas. See [Extension data types](ecosystem.ipynb#ecosystem-extensions) for a list of third-party
libraries that have implemented an extension.

The following table lists all of pandas extension types. See the respective
documentation sections for more on each type.

``````[](user_guide/timeseries.ipynb#timeseries-timezone)``````````[](user_guide/timeseries.ipynb#timeseries-periods)``````````[](user_guide/advanced.ipynb#advanced-intervalindex)````[](user_guide/integer_na.ipynb#integer-na)|Kind of Data|Data Type|Scalar|Array|Documentation|
|:-----------------:|:-----------------------:|:----------------:|:---------------------------:|:---------------------------:|
|tz-aware datetime|DatetimeTZDtype|Timestamp|arrays.DatetimeArray|Time zone handling|
|Categorical|CategoricalDtype|(none)|Categorical|categorical|
|period (time spans)|PeriodDtype|Period|arrays.PeriodArray|Time span representation|
|sparse|SparseDtype|(none)|arrays.SparseArray|sparse|
|intervals|IntervalDtype|Interval|arrays.IntervalArray|IntervalIndex|
|nullable integer|Int64Dtype, …|(none)|arrays.IntegerArray|Nullable integer data type|
Pandas uses the `object` dtype for storing strings.

Finally, arbitrary objects may be stored using the `object` dtype, but should
be avoided to the extent possible (for performance and interoperability with
other libraries and methods. See [object conversion](#basics-object-conversion)).

A convenient `dtypes` attribute for DataFrame returns a Series
with the data type of each column.

In [ ]:
dft = pd.DataFrame({'A': np.random.rand(3),
                    'B': 1,
                    'C': 'foo',
                    'D': pd.Timestamp('20010102'),
                    'E': pd.Series([1.0] * 3).astype('float32'),
                    'F': False,
                    'G': pd.Series([1] * 3, dtype='int8')})
dft
dft.dtypes

On a `Series` object, use the `dtype` attribute.

In [ ]:
dft['A'].dtype

If a pandas object contains data with multiple dtypes *in a single column*, the
dtype of the column will be chosen to accommodate all of the data types
(`object` is the most general).

In [ ]:
# these ints are coerced to floats
pd.Series([1, 2, 3, 4, 5, 6.])

# string data forces an ``object`` dtype
pd.Series([1, 2, 3, 6., 'foo'])

The number of columns of each type in a `DataFrame` can be found by calling
`DataFrame.dtypes.value_counts()`.

In [ ]:
dft.dtypes.value_counts()

Numeric dtypes will propagate and can coexist in DataFrames.
If a dtype is passed (either directly via the `dtype` keyword, a passed `ndarray`,
or a passed `Series`, then it will be preserved in DataFrame operations. Furthermore,
different numeric dtypes will **NOT** be combined. The following example will give you a taste.

In [ ]:
df1 = pd.DataFrame(np.random.randn(8, 1), columns=['A'], dtype='float32')
df1
df1.dtypes
df2 = pd.DataFrame({'A': pd.Series(np.random.randn(8), dtype='float16'),
                    'B': pd.Series(np.random.randn(8)),
                    'C': pd.Series(np.array(np.random.randn(8),
                                            dtype='uint8'))})
df2
df2.dtypes

### defaults

By default integer types are `int64` and float types are `float64`,
*regardless* of platform (32-bit or 64-bit).
The following will all result in `int64` dtypes.

In [ ]:
pd.DataFrame([1, 2], columns=['a']).dtypes
pd.DataFrame({'a': [1, 2]}).dtypes
pd.DataFrame({'a': 1}, index=list(range(2))).dtypes

Note that Numpy will choose *platform-dependent* types when creating arrays.
The following **WILL** result in `int32` on 32-bit platform.

In [ ]:
frame = pd.DataFrame(np.array([1, 2]))

### upcasting

Types can potentially be *upcasted* when combined with other types, meaning they are promoted
from the current type (e.g. `int` to `float`).

In [ ]:
df3 = df1.reindex_like(df2).fillna(value=0.0) + df2
df3
df3.dtypes

`DataFrame.to_numpy()` will return the *lower-common-denominator* of the dtypes, meaning
the dtype that can accommodate **ALL** of the types in the resulting homogeneous dtyped NumPy array. This can
force some *upcasting*.

In [ ]:
df3.to_numpy().dtype

### astype


<a id='basics-cast'></a>
You can use the `astype()` method to explicitly convert dtypes from one to another. These will by default return a copy,
even if the dtype was unchanged (pass `copy=False` to change this behavior). In addition, they will raise an
exception if the astype operation is invalid.

Upcasting is always according to the **numpy** rules. If two different dtypes are involved in an operation,
then the more *general* one will be used as the result of the operation.

In [ ]:
df3
df3.dtypes

# conversion of dtypes
df3.astype('float32').dtypes

Convert a subset of columns to a specified type using `astype()`.

In [ ]:
dft = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]})
dft[['a', 'b']] = dft[['a', 'b']].astype(np.uint8)
dft
dft.dtypes

New in version 0.19.0.

Convert certain columns to a specific dtype by passing a dict to `astype()`.

In [ ]:
dft1 = pd.DataFrame({'a': [1, 0, 1], 'b': [4, 5, 6], 'c': [7, 8, 9]})
dft1 = dft1.astype({'a': np.bool, 'c': np.float64})
dft1
dft1.dtypes

>**Note**
>
>When trying to convert a subset of columns to a specified type using `astype()`  and `loc()`, upcasting occurs.

`loc()` tries to fit in what we are assigning to the current dtypes, while `[]` will overwrite them taking the dtype from the right hand side. Therefore the following piece of code produces the unintended result.

In [ ]:
dft = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]})
dft.loc[:, ['a', 'b']].astype(np.uint8).dtypes
dft.loc[:, ['a', 'b']] = dft.loc[:, ['a', 'b']].astype(np.uint8)
dft.dtypes


<a id='basics-object-conversion'></a>

### object conversion

pandas offers various functions to try to force conversion of types from the `object` dtype to other types.
In cases where the data is already of the correct type, but stored in an `object` array, the
`DataFrame.infer_objects()` and `Series.infer_objects()` methods can be used to soft convert
to the correct type.

> 

In [ ]:
import datetime
df = pd.DataFrame([[1, 2],
                   ['a', 'b'],
                   [datetime.datetime(2016, 3, 2),
                    datetime.datetime(2016, 3, 2)]])
df = df.T
df
df.dtypes


Because the data was transposed the original inference stored all columns as object, which
`infer_objects` will correct.

> 

In [ ]:
df.infer_objects().dtypes


The following functions are available for one dimensional object arrays or scalars to perform
hard conversion of objects to a specified type:

- `to_numeric()` (conversion to numeric dtypes)  

In [ ]:
m = ['1.1', 2, 3]
pd.to_numeric(m)

- `to_datetime()` (conversion to datetime objects)  

In [ ]:
import datetime
m = ['2016-07-09', datetime.datetime(2016, 3, 2)]
pd.to_datetime(m)

- `to_timedelta()` (conversion to timedelta objects)  

In [ ]:
m = ['5us', pd.Timedelta('1day')]
pd.to_timedelta(m)



To force a conversion, we can pass in an `errors` argument, which specifies how pandas should deal with elements
that cannot be converted to desired dtype or object. By default, `errors='raise'`, meaning that any errors encountered
will be raised during the conversion process. However, if `errors='coerce'`, these errors will be ignored and pandas
will convert problematic elements to `pd.NaT` (for datetime and timedelta) or `np.nan` (for numeric). This might be
useful if you are reading in data which is mostly of the desired dtype (e.g. numeric, datetime), but occasionally has
non-conforming elements intermixed that you want to represent as missing:

In [ ]:
import datetime
m = ['apple', datetime.datetime(2016, 3, 2)]
pd.to_datetime(m, errors='coerce')

m = ['apple', 2, 3]
pd.to_numeric(m, errors='coerce')

m = ['apple', pd.Timedelta('1day')]
pd.to_timedelta(m, errors='coerce')

The `errors` parameter has a third option of `errors='ignore'`, which will simply return the passed in data if it
encounters any errors with the conversion to a desired data type:

In [ ]:
import datetime
m = ['apple', datetime.datetime(2016, 3, 2)]
pd.to_datetime(m, errors='ignore')

m = ['apple', 2, 3]
pd.to_numeric(m, errors='ignore')

m = ['apple', pd.Timedelta('1day')]
pd.to_timedelta(m, errors='ignore')

In addition to object conversion, `to_numeric()` provides another argument `downcast`, which gives the
option of downcasting the newly (or already) numeric data to a smaller dtype, which can conserve memory:

In [ ]:
m = ['1', 2, 3]
pd.to_numeric(m, downcast='integer')   # smallest signed int dtype
pd.to_numeric(m, downcast='signed')    # same as 'integer'
pd.to_numeric(m, downcast='unsigned')  # smallest unsigned int dtype
pd.to_numeric(m, downcast='float')     # smallest float dtype

As these methods apply only to one-dimensional arrays, lists or scalars; they cannot be used directly on multi-dimensional objects such
as DataFrames. However, with `apply()`, we can “apply” the function over each column efficiently:

In [ ]:
import datetime
df = pd.DataFrame([
    ['2016-07-09', datetime.datetime(2016, 3, 2)]] * 2, dtype='O')
df
df.apply(pd.to_datetime)

df = pd.DataFrame([['1.1', 2, 3]] * 2, dtype='O')
df
df.apply(pd.to_numeric)

df = pd.DataFrame([['5us', pd.Timedelta('1day')]] * 2, dtype='O')
df
df.apply(pd.to_timedelta)

### gotchas

Performing selection operations on `integer` type data can easily upcast the data to `floating`.
The dtype of the input data will be preserved in cases where `nans` are not introduced.
See also [Support for integer NA](user_guide/gotchas.ipynb#gotchas-intna).

In [ ]:
dfi = df3.astype('int32')
dfi['E'] = 1
dfi
dfi.dtypes

casted = dfi[dfi > 0]
casted
casted.dtypes

While float dtypes are unchanged.

In [ ]:
dfa = df3.copy()
dfa['A'] = dfa['A'].astype('float32')
dfa.dtypes

casted = dfa[df2 > 0]
casted
casted.dtypes

## Selecting columns based on `dtype`


<a id='basics-selectdtypes'></a>
The `select_dtypes()` method implements subsetting of columns
based on their `dtype`.

First, let’s create a `DataFrame` with a slew of different
dtypes:

In [ ]:
df = pd.DataFrame({'string': list('abc'),
                   'int64': list(range(1, 4)),
                   'uint8': np.arange(3, 6).astype('u1'),
                   'float64': np.arange(4.0, 7.0),
                   'bool1': [True, False, True],
                   'bool2': [False, True, False],
                   'dates': pd.date_range('now', periods=3),
                   'category': pd.Series(list("ABC")).astype('category')})
df['tdeltas'] = df.dates.diff()
df['uint64'] = np.arange(3, 6).astype('u8')
df['other_dates'] = pd.date_range('20130101', periods=3)
df['tz_aware_dates'] = pd.date_range('20130101', periods=3, tz='US/Eastern')
df

And the dtypes:

In [ ]:
df.dtypes

`select_dtypes()` has two parameters `include` and `exclude` that allow you to
say “give me the columns *with* these dtypes” (`include`) and/or “give the
columns *without* these dtypes” (`exclude`).

For example, to select `bool` columns:

In [ ]:
df.select_dtypes(include=[bool])

You can also pass the name of a dtype in the [NumPy dtype hierarchy](https://docs.scipy.org/doc/numpy/reference/arrays.scalars.html):

In [ ]:
df.select_dtypes(include=['bool'])

`select_dtypes()` also works with generic dtypes as well.

For example, to select all numeric and boolean columns while excluding unsigned
integers:

In [ ]:
df.select_dtypes(include=['number', 'bool'], exclude=['unsignedinteger'])

To select string columns you must use the `object` dtype:

In [ ]:
df.select_dtypes(include=['object'])

To see all the child dtypes of a generic `dtype` like `numpy.number` you
can define a function that returns a tree of child dtypes:

In [ ]:
def subdtypes(dtype):
    subs = dtype.__subclasses__()
    if not subs:
        return dtype
    return [dtype, [subdtypes(dt) for dt in subs]]

All NumPy dtypes are subclasses of `numpy.generic`:

In [ ]:
subdtypes(np.generic)

>**Note**
>
>Pandas also defines the types `category`, and `datetime64[ns, tz]`, which are not integrated into the normal
NumPy hierarchy and won’t show up with the above function.